# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#Data Pre-Processing Functions 

In [4]:
title_dict = {1:'alls-well-that-ends-well_TXT_FolgerShakespeare', 2:'a-midsummer-nights-dream_TXT_FolgerShakespeare', 3:'antony-and-cleopatra_TXT_FolgerShakespeare', 4:'as-you-like-it_TXT_FolgerShakespeare', 5:'coriolanus_TXT_FolgerShakespeare', 6:'cymbeline_TXT_FolgerShakespeare', 7:'hamlet_TXT_FolgerShakespeare', 8:'henry-iv-part-1_TXT_FolgerShakespeare', 9:'henry-iv-part-2_TXT_FolgerShakespeare',
              10:'henry-v_TXT_FolgerShakespeare', 11:'henry-viii_TXT_FolgerShakespeare', 12:'henry-vi-part-1_TXT_FolgerShakespeare', 13:'henry-vi-part-2_TXT_FolgerShakespeare',
              14:'henry-vi-part-3_TXT_FolgerShakespeare', 15:'julius-caesar_TXT_FolgerShakespeare', 16:'king-john_TXT_FolgerShakespeare', 17:'king-lear_TXT_FolgerShakespeare', 18:'loves-labors-lost_TXT_FolgerShakespeare', 19:'lucrece_TXT_FolgerShakespeare', 20:'lucrece_TXT_FolgerShakespeare', 21:'measure-for-measure_TXT_FolgerShakespeare', 22:'much-ado-about-nothing_TXT_FolgerShakespeare',
              23:'othello_TXT_FolgerShakespeare', 24:'pericles_TXT_FolgerShakespeare', 25:'richard-ii_TXT_FolgerShakespeare', 26:'richard-iii_TXT_FolgerShakespeare', 27:'romeo-and-juliet_TXT_FolgerShakespeare',
              28:'shakespeares-sonnets_TXT_FolgerShakespeare', 29:'the-comedy-of-errors_TXT_FolgerShakespeare', 30:'the-merchant-of-venice_TXT_FolgerShakespeare', 31:'the-merry-wives-of-windsor_TXT_FolgerShakespeare', 32:'the-phoenix-and-turtle_TXT_FolgerShakespeare', 33:'the-taming-of-the-shrew_TXT_FolgerShakespeare', 34:'the-tempest_TXT_FolgerShakespeare', 35:'the-two-gentlemen-of-verona_TXT_FolgerShakespeare', 36:'the-two-noble-kinsmen_TXT_FolgerShakespeare',
              37:'the-winters-tale_TXT_FolgerShakespeare', 38:'timon-of-athens_TXT_FolgerShakespeare', 39:'titus-andronicus_TXT_FolgerShakespeare', 40:'troilus-and-cressida_TXT_FolgerShakespeare', 41:'twelfth-night_TXT_FolgerShakespeare', 42:'venus-and-adonis_TXT_FolgerShakespeare'}

###Case Folding

In [5]:
def Case_Folding(data):
  data = data.lower()
  return data

### Remove Punctuations

In [6]:
def Remove_Punctuations(data):
  punc_list = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

  for i in data:
    if i in punc_list:
        data = data.replace(i, "")

  return data  

### Stop Word Removal

In [7]:
#List all stop words
#print(stopwords.words('english'))

In [8]:
#Removing Stopwords
def Stop_Word_Removal(data):
  stop_words = set(stopwords.words('english')) #Load above list of stop words 

  t = word_tokenize(data) #Tokenize the sentence. t=list of tokens in the sentence

  clean_data = [w for w in t if not w.lower() in stop_words] #Case folding for the stop words. 
  clean_data = [] #Stop word removed sentence
  
  for w in t:
      if w not in stop_words:
          clean_data.append(w)
  
  return clean_data

###Stemming

In [9]:
def Stemming_fn(data):
  ps = PorterStemmer()
  temp = []
  for i in data:
    temp.append(ps.stem(i))
  return temp

###Create Vocabulary of Unique Words

In [10]:
global_vocab = [] #Stores list of all words in the entire corpus (all docs)

In [11]:
#Since we arent working with phrase queries, we dont need to store multiple occurences of the same word. 
def save_unique_words(data):
  vocab = []
  for i in data:
    if i not in vocab: #Stores unique words for that document - used to build the posting lists
      vocab.append(i)
    if i not in global_vocab: #Stores unique words for the entire corpus 
      global_vocab.append(i)
  return vocab

# Function to Build a Posting List

In [12]:
#Create a posting list
def create_posting_list(data_dictionary):
  #data_dictionary - Key = doc number, value = list of words in the doc
  postings = {} #Posting list is a dictionary. Key = word, value = list of doc indices. 
  for key in data_dictionary: #For each document
    doc_data = data_dictionary[key] #Stores all unique words in that document
    for word in doc_data: #For each word in the document
      if word not in postings: 
        postings[word] = []
        postings[word].append(key)
      else:
        postings[word].append(key)
    
  return postings

#Edit Distance

In [13]:
def EditDistDP(str1, str2):
     
    len1 = len(str1)
    len2 = len(str2)
 
    
    DP = [[0 for i in range(len1 + 1)]
             for j in range(2)]; # DP array for memoization

 
    # Base case - 2nd string is empty
    for i in range(0, len1 + 1):
        DP[0][i] = i
 
    for i in range(1, len2 + 1): #For every character in 2nd string
        for j in range(0, len1 + 1): #Compare this char with str1 characters

            if (j == 0):
                DP[i % 2][j] = i #Base case - str1 is empty
 
            elif(str1[j - 1] == str2[i-1]):
                DP[i % 2][j] = DP[(i - 1) % 2][j - 1] #Characters match => Dont do anything
             
            else:
                DP[i % 2][j] = (1 + min(DP[(i - 1) % 2][j],
                                    min(DP[i % 2][j - 1],
                                  DP[(i - 1) % 2][j - 1])))
             

    return(DP[len2 % 2][len1]) #%2 since we end up in 0th/1st row depending on whether len2 is even/odd

#Boolean Information Retreival Function

In [14]:
def compute_documents(inp):
  closest_distance = 100000
  closest_word = ""
  l1 = [*range(1, 46)] #Final list
  l2 = [] #Temp list for each word
  temp = "" #Stores latest Boolean operation

  inp = Case_Folding(inp)
  inp = Remove_Punctuations(inp)
  res = inp.split()
  res = Stemming_fn(res)
  #print(res)

  for word in res:
    #First check if the word has correct spelling, i.e., it is a part of our vocabulary 
    if word not in global_vocab and word!="and" and word!="or" and word!="not":
      print("Possible Spelling Error. Matching with the closest word using least edit distance: ")
      for i in global_vocab:
        d = EditDistDP(word, i)
        if d<closest_distance:
          closest_distance = d
          closest_word = i
      print(closest_distance, closest_word)
      l2 = postings[closest_word]
      if temp=="":
        l1 = set(l1).intersection(l2) #Before encountering first boolean operator
      elif temp == "and":
        l1 = set(l1).intersection(l2)
      elif temp == "or":
        l1 = set(l1).union(l2)
      else: 
        l1 = set(l1).difference(l2)
      closest_distance = 100000 #Reset the values for next spelling mistake
      closest_word = "" #Reset the values for next spelling mistake


    else:
      if word == "and":
        temp = "and"
      elif word == "or":
        temp = "or"
      elif word == "not":
        temp = "not"
      else: #The word is valid and is not a boolean operator
        l2 = postings[word]
        if temp=="":
          l1 = set(l1).intersection(l2) #Before encountering first boolean operator
        elif temp == "and":
          l1 = set(l1).intersection(l2)
        elif temp == "or":
          l1 = set(l1).union(l2)
        else: 
          l1 = set(l1).difference(l2)
  print("The document numbers retreived are: ")
  print(l1)
  print("The document names retreived are: ")
  for i in l1: 
    print(title_dict[i])

#Dataset Loading 




In [15]:
path = "/content/drive/MyDrive/shakespeares-works_TXT_FolgerShakespeare/"

In [16]:
data_dictionary = {}
for i in range(1, 43):
    print(i)
    f = open(path+str(i)+".txt", 'r')
    data = f.read()
    data = Case_Folding(data)
    data = Remove_Punctuations(data)
    data = Stop_Word_Removal(data)
    data = Stemming_fn(data)
    data = save_unique_words(data)
    data_dictionary[i] = data #Make a dictionary. Key = doc number, value = list of words in the doc

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [17]:
#Call Function to create a posting list
postings = create_posting_list(data_dictionary)

In [18]:
postings

{'all': [1,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  13,
  14,
  17,
  20,
  22,
  23,
  26,
  34,
  36,
  37,
  38,
  40],
 'well': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42],
 'end': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42],
 'william': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42],
 'shakespear': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  1

In [19]:
inp = "avail and bond" 
compute_documents(inp)

The document numbers retreived are: 
{1, 37, 12, 19, 21}
The document names retreived are: 
alls-well-that-ends-well_TXT_FolgerShakespeare
the-winters-tale_TXT_FolgerShakespeare
henry-vi-part-1_TXT_FolgerShakespeare
lucrece_TXT_FolgerShakespeare
measure-for-measure_TXT_FolgerShakespeare


In [20]:
inp = "ador or sun"
compute_documents(inp)

The document numbers retreived are: 
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42}
The document names retreived are: 
alls-well-that-ends-well_TXT_FolgerShakespeare
a-midsummer-nights-dream_TXT_FolgerShakespeare
antony-and-cleopatra_TXT_FolgerShakespeare
as-you-like-it_TXT_FolgerShakespeare
coriolanus_TXT_FolgerShakespeare
cymbeline_TXT_FolgerShakespeare
hamlet_TXT_FolgerShakespeare
henry-iv-part-1_TXT_FolgerShakespeare
henry-iv-part-2_TXT_FolgerShakespeare
henry-v_TXT_FolgerShakespeare
henry-viii_TXT_FolgerShakespeare
henry-vi-part-1_TXT_FolgerShakespeare
henry-vi-part-2_TXT_FolgerShakespeare
henry-vi-part-3_TXT_FolgerShakespeare
julius-caesar_TXT_FolgerShakespeare
king-john_TXT_FolgerShakespeare
king-lear_TXT_FolgerShakespeare
loves-labors-lost_TXT_FolgerShakespeare
lucrece_TXT_FolgerShakespeare
lucrece_TXT_FolgerShakespeare
measure-for-measure_TXT_FolgerShakespeare
much-ado-

In [21]:
inp = "azail and bknd"
compute_documents(inp)

Possible Spelling Error. Matching with the closest word using least edit distance: 
1 avail
Possible Spelling Error. Matching with the closest word using least edit distance: 
1 bond
The document numbers retreived are: 
{1, 37, 12, 19, 21}
The document names retreived are: 
alls-well-that-ends-well_TXT_FolgerShakespeare
the-winters-tale_TXT_FolgerShakespeare
henry-vi-part-1_TXT_FolgerShakespeare
lucrece_TXT_FolgerShakespeare
measure-for-measure_TXT_FolgerShakespeare


In [22]:
inp = "worship NOT encount"
compute_documents(inp)

The document numbers retreived are: 
{2, 19, 28, 30}
The document names retreived are: 
a-midsummer-nights-dream_TXT_FolgerShakespeare
lucrece_TXT_FolgerShakespeare
shakespeares-sonnets_TXT_FolgerShakespeare
the-merchant-of-venice_TXT_FolgerShakespeare


In [23]:
inp = "worshipzz NOT enocount"
compute_documents(inp)

Possible Spelling Error. Matching with the closest word using least edit distance: 
2 worship
Possible Spelling Error. Matching with the closest word using least edit distance: 
1 encount
The document numbers retreived are: 
{2, 19, 28, 30}
The document names retreived are: 
a-midsummer-nights-dream_TXT_FolgerShakespeare
lucrece_TXT_FolgerShakespeare
shakespeares-sonnets_TXT_FolgerShakespeare
the-merchant-of-venice_TXT_FolgerShakespeare


In [24]:
inp = "anchoxd AND error AND chast" 
compute_documents(inp)

Possible Spelling Error. Matching with the closest word using least edit distance: 
2 anchor
The document numbers retreived are: 
{37, 6, 7, 21, 24, 28}
The document names retreived are: 
the-winters-tale_TXT_FolgerShakespeare
cymbeline_TXT_FolgerShakespeare
hamlet_TXT_FolgerShakespeare
measure-for-measure_TXT_FolgerShakespeare
pericles_TXT_FolgerShakespeare
shakespeares-sonnets_TXT_FolgerShakespeare


In [25]:
"""
'speedili': [1, 6, 8, 11, 14, 17, 21, 36]
'happen': [1, 7, 11, 12, 14, 22, 25, 33, 34, 36, 38]
'honestli': [1, 10, 22, 27, 38]
'likelihood': [1, 4, 6, 7, 8, 9, 10, 21, 22, 23, 26, 33, 35, 36]
"""

"\n'speedili': [1, 6, 8, 11, 14, 17, 21, 36]\n'happen': [1, 7, 11, 12, 14, 22, 25, 33, 34, 36, 38]\n'honestli': [1, 10, 22, 27, 38]\n'likelihood': [1, 4, 6, 7, 8, 9, 10, 21, 22, 23, 26, 33, 35, 36]\n"

In [26]:
inp = "speedily AND happened OR honestly" 
#Speedily and happened = 1, 11, 14, 36. 'OR' this with 1,10,22,27,38
compute_documents(inp)

The document numbers retreived are: 
{1, 36, 38, 10, 11, 14, 22, 27}
The document names retreived are: 
alls-well-that-ends-well_TXT_FolgerShakespeare
the-two-noble-kinsmen_TXT_FolgerShakespeare
timon-of-athens_TXT_FolgerShakespeare
henry-v_TXT_FolgerShakespeare
henry-viii_TXT_FolgerShakespeare
henry-vi-part-3_TXT_FolgerShakespeare
much-ado-about-nothing_TXT_FolgerShakespeare
romeo-and-juliet_TXT_FolgerShakespeare


In [27]:
inp = "speedily OR honestly Not happened" 
#Speedily or honestly = 1, 36, 6, 38, 8, 10, 11, 14, 17, 21, 22, 27. From this remove 1, 7, 11, 12, 14, 22, 25, 33, 34, 36, 38
compute_documents(inp)

The document numbers retreived are: 
{6, 8, 10, 17, 21, 27}
The document names retreived are: 
cymbeline_TXT_FolgerShakespeare
henry-iv-part-1_TXT_FolgerShakespeare
henry-v_TXT_FolgerShakespeare
king-lear_TXT_FolgerShakespeare
measure-for-measure_TXT_FolgerShakespeare
romeo-and-juliet_TXT_FolgerShakespeare


In [28]:
inp = "speedily or happened or honestly and likelihood" 
compute_documents(inp)

The document numbers retreived are: 
{1, 33, 36, 6, 7, 8, 10, 21, 22}
The document names retreived are: 
alls-well-that-ends-well_TXT_FolgerShakespeare
the-taming-of-the-shrew_TXT_FolgerShakespeare
the-two-noble-kinsmen_TXT_FolgerShakespeare
cymbeline_TXT_FolgerShakespeare
hamlet_TXT_FolgerShakespeare
henry-iv-part-1_TXT_FolgerShakespeare
henry-v_TXT_FolgerShakespeare
measure-for-measure_TXT_FolgerShakespeare
much-ado-about-nothing_TXT_FolgerShakespeare


#WildCard Queries - Bigram Indies

In [29]:
def create_bigram_list(word):
  l = []
  for i in range(0, len(word)-1):
    l.append(word[i]+word[i+1])
  return l

In [30]:
reverse_words = []
for i in global_vocab:
  temp = i[::-1] #reverse the word
  reverse_words.append(temp) 

#Build vocab for reverse+proper words
reverse_added_global_vocab = global_vocab + reverse_words

In [31]:
def bigram_save_unique_words(data):
  vocab = []
  reverse_words = []
  for word in data:
    if word_tokenize not in vocab:
      #Add the word, its reverse and compute the bigrams. 
      vocab.append(word)
      temp = word[::-1]
      vocab.append(temp)

      for i in range(0, len(word)-1):
        bi_g = word[i]+word[i+1]
        if bi_g not in vocab:
          vocab.append(bi_g)

      for i in range(0, len(temp)-1):
        bi_g = temp[i]+temp[i+1]
        if bi_g not in vocab:
          vocab.append(bi_g)

  return vocab

In [32]:
data_dictionary = {}
for i in range(1, 43):
  print(i)
  f = open(path+str(i)+".txt", 'r')
  data = f.read()
  data = Case_Folding(data)
  data = Remove_Punctuations(data)
  data = Stop_Word_Removal(data)
  data = Stemming_fn(data)
  data = bigram_save_unique_words(data)
  data_dictionary[i] = data #Make a dictionary. Key = doc number, value = list of words in the doc

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [33]:
postings = create_posting_list(data_dictionary)

In [34]:
len(postings)

40164

In [35]:
#Discard duplicacies - store only once 
for key in postings:
  l = postings[key]
  l = set(l)
  l = list(l)
  postings[key] = l

In [36]:
postings

{'all': [1,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  13,
  14,
  17,
  20,
  22,
  23,
  26,
  34,
  36,
  37,
  38,
  40],
 'lla': [1,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  13,
  14,
  17,
  20,
  22,
  23,
  26,
  34,
  36,
  37,
  38,
  40],
 'al': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42],
 'll': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42],
 'la': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,

In [37]:
def wildcard_compute_documents(inp):
  l1 = [*range(1, 46)] #Final list
  l2 = [] #Temp list for each word

  res = Case_Folding(inp)

  #Split m*n into two queries - m*, *n
  w1 = ""
  w2 = ""
  ct = 0 #Position of *
  for i in res: 
    if i!= "*":
      ct = ct+1
    else:
      break
  
  w1 = res[0:ct]
  w2 = res[ct+1:]
  w2 = w2[::-1] #Reverse of the word after *

  #Build bi-gram lists for w1, w2. 
  bgl1 = create_bigram_list(w1) #List to store all bigrams of w1
  bgl2 = create_bigram_list(w2) #List to store all bigrams of w2


  #Retreive posting lists for each bigram and take intersection for all. 
  for i in bgl1:
    l2 = postings[i]
    l1 = set(l1).intersection(l2)
  for i in bgl2:
    l2 = postings[i]
    l1 = set(l1).intersection(l2)    

  print("The document numbers retreived are: ")
  print(l1)
  print("The document names retreived are: ")
  for i in l1: 
    print(title_dict[i])

In [38]:
inp = "spee*ly" #Almost all docs will be retreived - disadvantage of using bigram indices. 
wildcard_compute_documents(inp)

The document numbers retreived are: 
{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 39, 40, 41, 42}
The document names retreived are: 
alls-well-that-ends-well_TXT_FolgerShakespeare
a-midsummer-nights-dream_TXT_FolgerShakespeare
antony-and-cleopatra_TXT_FolgerShakespeare
as-you-like-it_TXT_FolgerShakespeare
coriolanus_TXT_FolgerShakespeare
cymbeline_TXT_FolgerShakespeare
hamlet_TXT_FolgerShakespeare
henry-iv-part-1_TXT_FolgerShakespeare
henry-iv-part-2_TXT_FolgerShakespeare
henry-v_TXT_FolgerShakespeare
henry-viii_TXT_FolgerShakespeare
henry-vi-part-1_TXT_FolgerShakespeare
henry-vi-part-2_TXT_FolgerShakespeare
henry-vi-part-3_TXT_FolgerShakespeare
julius-caesar_TXT_FolgerShakespeare
king-john_TXT_FolgerShakespeare
king-lear_TXT_FolgerShakespeare
loves-labors-lost_TXT_FolgerShakespeare
lucrece_TXT_FolgerShakespeare
lucrece_TXT_FolgerShakespeare
measure-for-measure_TXT_FolgerShakespeare
much-ado-about-no

In [39]:
inp = "speedily"
compute_documents(inp)

The document numbers retreived are: 
{1, 36, 6, 8, 11, 14, 17, 21}
The document names retreived are: 
alls-well-that-ends-well_TXT_FolgerShakespeare
the-two-noble-kinsmen_TXT_FolgerShakespeare
cymbeline_TXT_FolgerShakespeare
henry-iv-part-1_TXT_FolgerShakespeare
henry-viii_TXT_FolgerShakespeare
henry-vi-part-3_TXT_FolgerShakespeare
king-lear_TXT_FolgerShakespeare
measure-for-measure_TXT_FolgerShakespeare


In [40]:
st = "park*spak"
w1 = ""
w2 = ""
ct = 0
for i in st: 
  if i!= "*":
    ct = ct+1
  else:
    break
print (ct)

4


In [41]:
w1 = st[0:ct]
w2 = st[ct+1:]
print(w1)
print(w2)

park
spak


In [42]:
p1 = create_bigram_list(w1)
p1

['pa', 'ar', 'rk']

In [43]:
p2 = create_bigram_list(w2)
p2

['sp', 'pa', 'ak']

#WildCard Queries - K gram Indices

In [44]:
inp = "aban*on" 
ct1 = 0
ct2 = 0 #Position of *
for i in inp: 
  if i!= "*":
    ct1 = ct1+1
  else:
    break

ct2 = len(inp)-ct1-1

In [45]:
print(ct1)
print(ct2)

4
2


In [46]:
def create_kgram_list(word, k):
  l = []
  for i in range(0, len(word)-k+1):
    l.append(word[i:i+k])
  return l

In [47]:
def kgram_save_unique_words(data, k):
  vocab = []
  reverse_words = []
  for word in data:
    if word_tokenize not in vocab:
      #Add the word, its reverse and compute the k-grams. 
      vocab.append(word)
      temp = word[::-1]
      vocab.append(temp)

      for i in range(0, len(word)-k+1):
        bi_g = word[i:i+k] 
        if bi_g not in vocab:
          vocab.append(bi_g)

      for i in range(0, len(temp)-k+1):
        bi_g = word[i:i+k]
        if bi_g not in vocab:
          vocab.append(bi_g)

  return vocab

In [48]:
data_dictionary = {}
for i in range(1, 43):
  print(i)
  f = open(path+str(i)+".txt", 'r')
  data = f.read()
  data = Case_Folding(data)
  data = Remove_Punctuations(data)
  data = Stop_Word_Removal(data)
  data = Stemming_fn(data)
  data = kgram_save_unique_words(data, ct1) #for part before *
  data = kgram_save_unique_words(data, ct2) #for part after *
  data_dictionary[i] = data #Make a dictionary. Key = doc number, value = list of words in the doc

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [49]:
postings = create_posting_list(data_dictionary)

In [50]:
len(postings)

72698

In [51]:
#Discard duplicacies - store only once 
for key in postings:
  l = postings[key]
  l = set(l)
  l = list(l)
  postings[key] = l

In [52]:
postings

{'all': [1,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  13,
  14,
  17,
  20,
  22,
  23,
  26,
  34,
  36,
  37,
  38,
  40],
 'lla': [1,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  13,
  14,
  17,
  20,
  22,
  23,
  26,
  34,
  36,
  37,
  38,
  40],
 'al': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42],
 'll': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42],
 'la': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,

In [53]:
def wildcard_compute_documents(inp):
  l1 = [] #Final list
  l2 = [] #Temp list for each word

  res = Case_Folding(inp)

  #Split m*n into two queries - m*, *n
  w1 = ""
  w2 = ""
  
  
  w1 = res[0:ct1]
  w2 = res[ct1+1:]
  w2 = w2[::-1]
  print(w1)
  print(w2)

  l1 = postings[w1]
  l2 = postings[w2]
  print("Posting list of w1: ")
  print(l1)
  print("Posting list of w2: ")
  print(l2)
  l3 = set(l1).intersection(l2)
  print("Posting list of wildcard query:")
  print(l3)

  print("The document names retreived are: ")
  for i in l3: 
    print(title_dict[i])
 

In [54]:
wildcard_compute_documents(inp)

aban
no
Posting list of w1: 
[1, 33, 4, 36, 38, 39, 40, 41, 10, 14, 18, 23, 30]
Posting list of w2: 
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42]
Posting list of wildcard query:
{1, 33, 4, 36, 38, 39, 40, 41, 10, 14, 18, 23, 30}
The document names retreived are: 
alls-well-that-ends-well_TXT_FolgerShakespeare
the-taming-of-the-shrew_TXT_FolgerShakespeare
as-you-like-it_TXT_FolgerShakespeare
the-two-noble-kinsmen_TXT_FolgerShakespeare
timon-of-athens_TXT_FolgerShakespeare
titus-andronicus_TXT_FolgerShakespeare
troilus-and-cressida_TXT_FolgerShakespeare
twelfth-night_TXT_FolgerShakespeare
henry-v_TXT_FolgerShakespeare
henry-vi-part-3_TXT_FolgerShakespeare
loves-labors-lost_TXT_FolgerShakespeare
othello_TXT_FolgerShakespeare
the-merchant-of-venice_TXT_FolgerShakespeare


In [55]:
postings["abandon"] #Observe that this list is a subset of aban*on - Better result as compared to 2 gram. 
#Disadvantage - Posting list has to be created for each input. 

[1, 33, 4, 36, 38, 39, 40, 41, 14, 23]

In [56]:
#Queries of form xyz* 
inp = "aban*" #1, 33, 4, 36, 38, 39, 40, 41, 10, 14, 18, 23, 30
inp = inp[:-1]
print(postings[inp])
print("The document names are: ")
for i in postings[inp]: 
    print(title_dict[i])

[1, 33, 4, 36, 38, 39, 40, 41, 10, 14, 18, 23, 30]
The document names are: 
alls-well-that-ends-well_TXT_FolgerShakespeare
the-taming-of-the-shrew_TXT_FolgerShakespeare
as-you-like-it_TXT_FolgerShakespeare
the-two-noble-kinsmen_TXT_FolgerShakespeare
timon-of-athens_TXT_FolgerShakespeare
titus-andronicus_TXT_FolgerShakespeare
troilus-and-cressida_TXT_FolgerShakespeare
twelfth-night_TXT_FolgerShakespeare
henry-v_TXT_FolgerShakespeare
henry-vi-part-3_TXT_FolgerShakespeare
loves-labors-lost_TXT_FolgerShakespeare
othello_TXT_FolgerShakespeare
the-merchant-of-venice_TXT_FolgerShakespeare


In [57]:
#Queries of form *xyz
inp = "*bund" #Eg: abund
#1, 34, 5, 37, 39, 8, 9, 40, 16, 24, 25, 28, 30
#Search posting list for dnub*
inp = inp[1:]
inp = inp [::-1]
print(postings[inp])
print("The document names are: ")
for i in postings[inp]: 
    print(title_dict[i])

[1, 34, 5, 37, 39, 8, 9, 40, 16, 24, 25, 28, 30]
The document names are: 
alls-well-that-ends-well_TXT_FolgerShakespeare
the-tempest_TXT_FolgerShakespeare
coriolanus_TXT_FolgerShakespeare
the-winters-tale_TXT_FolgerShakespeare
titus-andronicus_TXT_FolgerShakespeare
henry-iv-part-1_TXT_FolgerShakespeare
henry-iv-part-2_TXT_FolgerShakespeare
troilus-and-cressida_TXT_FolgerShakespeare
king-john_TXT_FolgerShakespeare
pericles_TXT_FolgerShakespeare
richard-ii_TXT_FolgerShakespeare
shakespeares-sonnets_TXT_FolgerShakespeare
the-merchant-of-venice_TXT_FolgerShakespeare


In [58]:
inp = "abund"
print(postings[inp]) #It is clearly evident that this list is a subset of the above list. 
print("The document names are: ")
for i in postings[inp]: 
    print(title_dict[i])

[1, 34, 5, 8, 9, 40, 16, 24, 25, 28, 30]
The document names are: 
alls-well-that-ends-well_TXT_FolgerShakespeare
the-tempest_TXT_FolgerShakespeare
coriolanus_TXT_FolgerShakespeare
henry-iv-part-1_TXT_FolgerShakespeare
henry-iv-part-2_TXT_FolgerShakespeare
troilus-and-cressida_TXT_FolgerShakespeare
king-john_TXT_FolgerShakespeare
pericles_TXT_FolgerShakespeare
richard-ii_TXT_FolgerShakespeare
shakespeares-sonnets_TXT_FolgerShakespeare
the-merchant-of-venice_TXT_FolgerShakespeare


#Combinging Wildcard and Boolean Queries

In [59]:
def compute_counts(inp):
  ct1 = 0
  ct2 = 0 #Position of *
  for i in inp: 
    if i!= "*":
      ct1 = ct1+1
    else:
      break

  ct2 = len(inp)-ct1-1

  return ct1, ct2

In [60]:
def wildcard_compute_documents(inp, ct1, ct2):
  l1 = [] #Final list
  l2 = [] #Temp list for each word

  res = Case_Folding(inp)

  #Split m*n into two queries - m*, *n
  w1 = ""
  w2 = ""
  
  
  w1 = res[0:ct1]
  w2 = res[ct1+1:]
  w2 = w2[::-1]
  print(w1)
  print(w2)

  l1 = postings[w1]
  l2 = postings[w2]
  
  l3 = set(l1).intersection(l2)
  return l3
 

In [61]:
#General Case - Combinging wilcard and boolean queries. 
def compute_documents(inp):
  closest_distance = 100000
  closest_word = ""
  l1 = [*range(1, 46)] #Final list
  l2 = [] #Temp list for each word
  temp = "" #Stores latest Boolean operation

  inp = Case_Folding(inp)
  res = inp.split()
  res = Stemming_fn(res)
  #print(res)

  for word in res:
    if '*' in word: 
      ct1, ct2 = compute_counts(word)
      l2 = wildcard_compute_documents(word, ct1, ct2)
      if temp=="":
          l1 = set(l1).intersection(l2) #Before encountering first boolean operator
      elif temp == "and":
          l1 = set(l1).intersection(l2)
      elif temp == "or":
          l1 = set(l1).union(l2)
      else: 
          l1 = set(l1).difference(l2)


    elif word not in global_vocab and word!="and" and word!="or" and word!="not":
      print("Possible Spelling Error. Matching with the closest word using least edit distance: ")
      for i in global_vocab:
        d = EditDistDP(word, i)
        if d<closest_distance:
          closest_distance = d
          closest_word = i
      print(closest_distance, closest_word)
      l2 = postings[closest_word]
      if temp=="":
        l1 = set(l1).intersection(l2) #Before encountering first boolean operator
      elif temp == "and":
        l1 = set(l1).intersection(l2)
      elif temp == "or":
        l1 = set(l1).union(l2)
      else: 
        l1 = set(l1).difference(l2)
      closest_distance = 100000 #Reset the values for next spelling mistake
      closest_word = "" #Reset the values for next spelling mistake


    else:
      if word == "and":
        temp = "and"
      elif word == "or":
        temp = "or"
      elif word == "not":
        temp = "not"
      else: 
        l2 = postings[word]
        if temp=="":
          l1 = set(l1).intersection(l2) #Before encountering first boolean operator
        elif temp == "and":
          l1 = set(l1).intersection(l2)
        elif temp == "or":
          l1 = set(l1).union(l2)
        else: 
          l1 = set(l1).difference(l2)
  print("The document numbers retreived are: ")
  print(l1)
  print("The document names retreived are: ")
  for i in l1: 
    print(title_dict[i])

In [62]:
inp = "aban*on and honestly AND Worshipzz"
#aban*on: 1, 33, 4, 36, 38, 39, 40, 41, 10, 14, 18, 23, 30
#honestli: 1, 10, 22, 27, 38
#worship: 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 40
#Intersection = 1, 10, 38
compute_documents(inp)

aban
no
Possible Spelling Error. Matching with the closest word using least edit distance: 
2 worship
The document numbers retreived are: 
{1, 10, 38}
The document names retreived are: 
alls-well-that-ends-well_TXT_FolgerShakespeare
henry-v_TXT_FolgerShakespeare
timon-of-athens_TXT_FolgerShakespeare
